In [104]:
import os
from dotenv import load_dotenv
import glob

import gradio as gr
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, convert_to_messages

In [2]:
load_dotenv(override=True)

True

In [35]:
# constants
LLM_MODEL = 'gpt-4.1-mini'
EMBEDDING_MODEL = "text-embedding-3-large"
BASE_DOCUMENT_FOLDER = 'knowledge-base/*'
VECTOR_DB_NAME = 'insurancellm_db'

In [18]:
document_folders = glob.glob(BASE_DOCUMENT_FOLDER, recursive=True)

docu_list = []

for folder in document_folders:
    data_type = os.path.basename(folder)

    document_list = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs={"encoding": "utf-8"})
    documents = document_list.load()

    for document in documents:
        document.metadata["data_type"] = data_type
        docu_list.append(document)

In [27]:
text_split = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200)

chunked_text = text_split.split_documents(docu_list)

In [38]:
embedding_function = OpenAIEmbeddings(model=EMBEDDING_MODEL)

if os.path.exists(VECTOR_DB_NAME):
    Chroma(persist_directory=VECTOR_DB_NAME, embedding_function=embedding_function).delete_collection()

vector_store = Chroma.from_documents(documents=chunked_text, embedding=embedding_function, persist_directory=VECTOR_DB_NAME)

print(f"total chunks added by the embedding function {vector_store._collection.count()}")

total chunks added by the embedding function 413


In [145]:
retriever = vector_store.as_retriever()
llm = ChatOpenAI(model=LLM_MODEL, temperature=0)

In [146]:
retriever.invoke("what is the salary of your ceo?")

[Document(id='7a47de21-69f1-4c02-9d29-43fddd84a929', metadata={'data_type': 'employees', 'source': 'knowledge-base/employees/Avery Lancasters.md'}, page_content='## Compensation History\n- **2015**: $150,000 base salary + Significant equity stake  \n- **2016**: $160,000 base salary + Equity increase  \n- **2017**: $150,000 base salary + Decrease in bonus due to performance  \n- **2018**: $180,000 base salary + performance bonus of $30,000  \n- **2019**: $185,000 base salary + market adjustment + $5,000 bonus  \n- **2020**: $170,000 base salary (temporary reduction due to COVID-19)  \n- **2021**: $200,000 base salary + performance bonus of $50,000  \n- **2022**: $210,000 base salary + retention bonus  \n- **2023**: $225,000 base salary + $75,000 performance bonus'),
 Document(id='9cd96ac9-563c-438f-a87c-a2497caba859', metadata={'source': 'knowledge-base/employees/James Wilson.md', 'data_type': 'employees'}, page_content='- **2017:** Rating: 4.3/5\n  *Strong start as CTO. Quickly assesse

In [83]:
SYSTEM_PROMPT_TEMPLATE = """
You are a knowledgeable, friendly assistant representing the company Insurellm.
You are chatting with a user about Insurellm.
If relevant, use the given context to answer any question.
If you don't know the answer, say so.
Context:
{context}
"""

In [156]:
def chat_with_vector_db(question, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]

    messages = []
    documents = retriever.invoke(question)

    context = [document.page_content for document in documents]
    joined_context = "\n\n---\n\n".join(context)
    print(f"joined context -> {joined_context}")
    system_prompt = SYSTEM_PROMPT_TEMPLATE.format(context=joined_context)

    system_message = SystemMessage(content=system_prompt)
    human_message = HumanMessage(content=question)

    messages.append(system_message)
    messages.extend(convert_to_messages(history))
    messages.append(human_message)

    response = llm.invoke(messages)

    return response.content

In [161]:
chat_with_vector_db("do you have any employees with expertise in machine learning?", [])

joined context -> ## Other HR Notes
- **Education:** PhD in Computer Science (Machine Learning) from Stanford University
- **Publications:** 7+ peer-reviewed papers in ML conferences and journals
- **Speaking:** Regular speaker at ML and InsurTech conferences. Represented Insurellm at 4 major conferences in 2023.
- **Patents:** Co-inventor on 2 pending patents for insurance ML applications
- **Recognition:** Data Science Excellence Award 2023, featured in InsureTech Innovation Magazine
- **Skills:** Expert in Python, TensorFlow, PyTorch, scikit-learn, SQL, and cloud ML platforms
- **Feedback:** World-class technical talent with strong business acumen. Natural leader who elevates entire team. Key retention priority.

---

**DevOps Engineer** - New York, NY
- Manage cloud infrastructure (AWS/GCP)
- Implement CI/CD pipelines and automation
- Monitor system performance and security
- 3+ years experience in DevOps/SRE

**Mobile Developer (iOS/Android)** - San Francisco, CA
- Build native mo

'Yes, we have employees with strong expertise in machine learning. For example, Priya Sharma is a Senior Data Scientist at Insurellm with a PhD in Computer Science specializing in machine learning. She leads machine learning initiatives for risk prediction models, has published multiple research papers on insurance ML applications, and has co-invented patents related to insurance ML. Additionally, she is skilled in Python, TensorFlow, PyTorch, and other ML tools, making her a key technical talent in our team. If you need more information about our machine learning experts or their projects, feel free to ask!'

In [162]:
gr.ChatInterface(fn=chat_with_vector_db).launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


joined context -> ## Other HR Notes
- **Education:** MS in Human Resources Management from Cornell University, BA in Psychology from UCLA
- **Certifications:** SHRM-SCP, Certified Diversity Professional
- **Recognition:** HR Excellence Award 2023 for diversity initiatives
- **Leadership:** Chairs Insurellm's Diversity & Inclusion committee
- **Skills:** Expert in talent acquisition, performance management, employee relations, and organizational development. Strong coach and advisor to leadership.
- **Feedback:** Trusted HR partner with excellent judgment and discretion. Strategic thinker who balances employee advocacy with business needs. Strong emotional intelligence.

---

Alex Chen continues to be a vital asset at Insurellm, contributing significantly to innovative backend solutions that help shape the future of insurance technology.

---

Claimllm represents the future of insurance claims—faster, smarter, and more customer-centric. Transform your claims operation and deliver the se